# மிஸ்ட்ரல் மாடல்களுடன் கட்டமைத்தல்

## அறிமுகம்

இந்த பாடத்தில் நீங்கள் கற்றுக்கொள்ளப்போகிறீர்கள்:  
- மிஸ்ட்ரல் மாடல்களின் பல்வேறு வகைகளை ஆராய்தல்  
- ஒவ்வொரு மாடலின் பயன்பாடுகள் மற்றும் சூழல்களைப் புரிந்துகொள்வது  
- ஒவ்வொரு மாடலின் தனித்துவமான அம்சங்களை காட்டும் குறியீட்டு உதாரணங்கள்.  


## மிஸ்ட்ரல் மாடல்கள்

இந்த பாடத்தில், நாங்கள் மிஸ்ட்ரல் மாடல்களின் 3 வெவ்வேறு வகைகளை ஆராய்வோம்:  
**மிஸ்ட்ரல் லார்ஜ்**, **மிஸ்ட்ரல் ஸ்மால்** மற்றும் **மிஸ்ட்ரல் நெமோ**.  

இந்த மாடல்கள் ஒவ்வொன்றும் Github Model சந்தையில் இலவசமாக கிடைக்கின்றன. இந்த நோட்புக்கில் உள்ள குறியீடு இந்த மாடல்களை பயன்படுத்தி செயல்படுத்தப்படும். AI மாடல்களுடன் [முன்மாதிரியாக்கம் செய்ய Github Models](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) பயன்படுத்துவது பற்றிய மேலும் விவரங்கள் இங்கே உள்ளன.


## மிஸ்ட்ரல் லார்ஜ் 2 (2407)
மிஸ்ட்ரல் லார்ஜ் 2 தற்போது மிஸ்ட்ரல் நிறுவனத்தின் முன்னணி மாடலாக உள்ளது மற்றும் நிறுவன பயன்பாட்டிற்காக வடிவமைக்கப்பட்டுள்ளது.

இந்த மாடல் மிஸ்ட்ரல் லார்ஜ் மாடலின் மேம்பட்ட பதிப்பாகும், இது வழங்குகிறது:
-  பெரிய கான்டெக்ஸ்ட் விண்டோ - 128k vs 32k 
-  கணித மற்றும் குறியீட்டு பணிகளில் மேம்பட்ட செயல்திறன் - சராசரி துல்லியம் 76.9% vs 60.4% 
-  பல மொழிகளில் மேம்பட்ட செயல்திறன் - மொழிகள்: ஆங்கிலம், பிரெஞ்சு, ஜெர்மன், ஸ்பானிஷ், இத்தாலியன், போர்ச்சுகீஸ், டச்சு, ரஷியன், சீனம், ஜப்பானியம், கொரியன், அரபு, மற்றும் இந்தி.

இந்த அம்சங்களுடன், மிஸ்ட்ரல் லார்ஜ் சிறப்பாக செயல்படுகிறது:
- *ரிட்ரீவல் ஆக்மென்டட் ஜெனரேஷன் (RAG)* - பெரிய கான்டெக்ஸ்ட் விண்டோவால்
- *ஃபங்ஷன் காலிங்* - இந்த மாடலில் இயல்பாகவே ஃபங்ஷன் காலிங் உள்ளது, இது வெளிப்புற கருவிகள் மற்றும் APIகளுடன் ஒருங்கிணைக்க அனுமதிக்கிறது. இந்த அழைப்புகள் ஒரே நேரத்தில் அல்லது தொடர்ச்சியாக ஒன்றின் பின் ஒன்றாக செய்யலாம்.
- *கோடு ஜெனரேஷன்* - இந்த மாடல் Python, Java, TypeScript மற்றும் C++ கோடு உருவாக்கத்தில் சிறப்பாக செயல்படுகிறது.


### RAG உதாரணம் Mistral Large 2 பயன்படுத்தி


இந்த எடுத்துக்காட்டில், Mistral Large 2 மாடலை பயன்படுத்தி ஒரு RAG முறைமையை ஒரு உரை ஆவணத்தில் இயக்குகிறோம். கேள்வி கொரிய மொழியில் எழுதப்பட்டுள்ளது மற்றும் கல்லூரிக்கு முன் ஆசிரியரின் செயல்பாடுகள் குறித்து கேட்கிறது.

இது Cohere Embeddings Model-ஐ பயன்படுத்தி உரை ஆவணத்தின் மற்றும் கேள்வியின் எம்பெடிங்குகளை உருவாக்குகிறது. இந்த மாதிரிக்காக, faiss Python package-ஐ ஒரு வெக்டர் சேமிப்பகமாக பயன்படுத்துகிறது.

Mistral மாடலுக்கு அனுப்பப்படும் ப்ராம்ப்டில் கேள்விகளும், கேள்விக்கு தொடர்புடையதாக இருக்கும் உரை துண்டுகளும் சேர்க்கப்பட்டுள்ளன. மாடல் பின்னர் ஒரு இயற்கை மொழி பதிலை வழங்குகிறது.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## மிஸ்ட்ரல் ஸ்மால்  
மிஸ்ட்ரல் ஸ்மால் என்பது மிஸ்ட்ரல் மாடல்களின் குடும்பத்தில் உள்ள மற்றொரு மாடல், இது பிரீமியர்/என்டர்பிரைஸ் வகையில் உள்ளது. பெயரிலேயே தெரிகின்றது போல, இந்த மாடல் ஒரு சிறிய மொழி மாடல் (SLM) ஆகும். மிஸ்ட்ரல் ஸ்மால் பயன்படுத்துவதன் நன்மைகள்:  
- மிஸ்ட்ரல் LLMs போன்ற மிஸ்ட்ரல் லார்ஜ் மற்றும் NeMo-வுடன் ஒப்பிடும்போது செலவுச்செலவு குறைவு - 80% விலை குறைவு  
- குறைந்த தாமதம் - மிஸ்ட்ரல் LLMs-களை விட வேகமான பதில்  
- நெகிழ்வானது - தேவையான வளங்கள் குறைவாகவே இருக்கும், பல சூழல்களில் பயன்படுத்தலாம்  

மிஸ்ட்ரல் ஸ்மால் சிறப்பாக பயன்படும் இடங்கள்:  
- சுருக்கம், உணர்வு பகுப்பாய்வு மற்றும் மொழிபெயர்ப்பு போன்ற உரை அடிப்படையிலான பணிகள்  
- அதன் செலவுச்செலவின் காரணமாக அடிக்கடி கோரிக்கைகள் செய்யப்படும் பயன்பாடுகள்  
- குறைந்த தாமதம் கொண்ட குறியீட்டு பணிகள், உதாரணமாக மதிப்பீடு மற்றும் குறியீட்டு பரிந்துரைகள்  


## Mistral Small மற்றும் Mistral Large-ஐ ஒப்பிடுதல்

Mistral Small மற்றும் Large இடையேயான தாமதங்களில் வேறுபாடுகளை காட்ட, கீழே உள்ள செறிவுகளை இயக்கவும்.

3-5 விநாடிகளுக்குள் பதிலளிக்கும் நேரங்களில் ஒரு வேறுபாட்டைக் காணலாம். அதேபோல், பதில்களின் நீளங்கள் மற்றும் பாணியை ஒரே கேள்வியில் கவனிக்கவும்.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## மிஸ்ட்ரல் நீமோ

இந்த பாடத்தில் விவாதிக்கப்பட்ட மற்ற இரண்டு மாதிரிகளுடன் ஒப்பிடும்போது, மிஸ்ட்ரல் நீமோ என்பது Apache2 உரிமத்துடன் கிடைக்கும் ஒரே இலவச மாதிரியாகும்.

மிஸ்ட்ரல் நிறுவனத்தின் முந்தைய திறந்த மூல LLM, மிஸ்ட்ரல் 7B-க்கு மேம்படுத்தலாக இது பார்க்கப்படுகிறது.

NeMo மாதிரியின் சில முக்கிய அம்சங்கள்:

- *மேம்பட்ட டோக்கனீசேஷன்:* இந்த மாதிரி பொதுவாக பயன்படுத்தப்படும் tiktoken ஐ விட Tekken டோக்கனைசரை பயன்படுத்துகிறது. இது பல மொழிகள் மற்றும் குறியீடுகளில் சிறந்த செயல்திறனை வழங்குகிறது.

- *பயிற்சி மேம்பாடு:* அடிப்படை மாதிரி பயிற்சி மேம்பாட்டுக்கு கிடைக்கிறது. இது பயிற்சி மேம்பாடு தேவைப்படும் பயன்பாடுகளுக்கு அதிக சுதந்திரத்தை வழங்குகிறது.

- *நேட்டிவ் ஃபங்ஷன் காலிங்:* மிஸ்ட்ரல் லார்ஜ் மாதிரியைப் போலவே, இந்த மாதிரி ஃபங்ஷன் காலிங்கில் பயிற்சி பெற்றுள்ளது. இது திறந்த மூல மாதிரிகளில் முதன்முதலில் இதைச் செய்யும் தனித்துவமானதாகும்.


## மிஸ்ட்ரல் நீமோ

இந்த பாடத்தில் விவாதிக்கப்பட்ட மற்ற இரண்டு மாதிரிகளுடன் ஒப்பிடும்போது, மிஸ்ட்ரல் நீமோ என்பது Apache2 உரிமத்துடன் கிடைக்கும் ஒரே இலவச மாதிரியாகும்.

இது மிஸ்ட்ரல் நிறுவனத்தின் முந்தைய திறந்த மூல LLM, மிஸ்ட்ரல் 7B-க்கு மேம்படுத்தலாக பார்க்கப்படுகிறது.

NeMo மாதிரியின் சில முக்கிய அம்சங்கள்:

- *மேம்பட்ட டோக்கனீசேஷன்:* இந்த மாதிரி பொதுவாக பயன்படுத்தப்படும் tiktoken ஐ விட Tekken tokenizer ஐ பயன்படுத்துகிறது. இது பல மொழிகள் மற்றும் குறியீடுகளில் சிறந்த செயல்திறனை வழங்குகிறது.

- *நுணுக்கமாக அமைத்தல்:* அடிப்படை மாதிரி நுணுக்கமாக அமைக்க கிடைக்கிறது. இது நுணுக்கமாக அமைத்தல் தேவைப்படும் பயன்பாடுகளுக்கு அதிக சாளரத்தன்மையை வழங்குகிறது.

- *உள்ளமைவான செயல்பாட்டு அழைப்பு:* மிஸ்ட்ரல் லார்ஜ் மாதிரியைப் போலவே, இந்த மாதிரி செயல்பாட்டு அழைப்பில் பயிற்சி பெற்றுள்ளது. இது இதை முதல் திறந்த மூல மாதிரிகளில் ஒன்றாக தனித்துவமாக ஆக்குகிறது.


### டோக்கனைசர்களை ஒப்பிடுதல்

இந்த மாதிரியில், Mistral NeMo டோக்கனைசேஷனை Mistral Large உடன் ஒப்பிட்டு எப்படி கையாளுகிறது என்பதை பார்க்கலாம்.

இரண்டு மாதிரிகளும் ஒரே ப்ராம்ப்டை எடுத்துக்கொள்கின்றன, ஆனால் NeMo Mistral Large-ஐ விட குறைவான டோக்கன்களை திருப்பி அளிக்கிறது என்பதை நீங்கள் காணலாம்.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## கற்றல் இங்கே நிற்கவில்லை, பயணத்தை தொடருங்கள்

இந்த பாடத்தை முடித்த பிறகு, உங்கள் Generative AI அறிவை மேலும் மேம்படுத்த [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) ஐ பாருங்கள்!



---

**அறிவிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையை பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கிறோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்கக்கூடும் என்பதை கவனத்தில் கொள்ளவும். அதன் சொந்த மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்களுக்கும் அல்லது தவறான விளக்கங்களுக்கும் நாங்கள் பொறுப்பல்ல.
